In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torchmetrics.classification import MulticlassF1Score
import CNNutils as utils
import torch.optim as optim
import torch.nn as nn
import os
import numpy as np
import matplotlib.pyplot as plt
from torchinfo import summary

In [2]:


# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=False, transform=transforms.ToTensor())

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=False, transform=transforms.ToTensor())

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
data_path='../0925/data/CIFAR/'
filelist=os.listdir(data_path)
trainlist=filelist[1:6]
trainlist

['data_batch_1',
 'data_batch_2',
 'data_batch_3',
 'data_batch_4',
 'data_batch_5']

In [5]:
for file in trainlist:
    aa=unpickle(data_path+file)
    # print(aa[b'data'][0])
    # print(aa[b'data'].reshape(10000,3,1024))
    data=aa[b'data'].reshape(-1,3,32,32)
    label=aa[b'labels']
    if file==trainlist[0]:
        feature=data
        labels=label
    else: 
        feature=np.concatenate([feature,data],axis=0)
        labels=labels+label
        
    print(feature.shape)
    print(len(labels))

(10000, 3, 32, 32)
10000
(20000, 3, 32, 32)
20000
(30000, 3, 32, 32)
30000
(40000, 3, 32, 32)
40000
(50000, 3, 32, 32)
50000


In [6]:
# plt.imshow(feature[0].transpose(2,1,0))
# plt.show()

In [7]:
# x=feature[0]
# print(x.size(0))
# print(x.shape[0])

In [8]:
test_file=filelist[-1]
test=unpickle(data_path+test_file)
test_feature=test[b'data'].reshape(-1,3,32,32)
test_label=test[b'labels']


In [9]:
trainset=utils.MyDataSet(feature,labels)
testset=utils.MyDataSet(test_feature,test_label)

In [10]:
# # 확인
# testDL=DataLoader(testset,batch_size=10)
# for feature,target in testDL:
#     #print(feature.shape)
#     feature=feature.permute(0,3,2,1)
#     print(feature.shape)
#     print(target)
#     break

In [11]:
Batch_size=64
trainDL=DataLoader(trainset,batch_size=Batch_size)
testDL=DataLoader(testset,batch_size=Batch_size)

In [12]:
model=utils.CNNmodel(3,len(classes),32,Knums=[16,32,64,128],Pnums=[64,32,16])
optimizer=optim.Adam(model.parameters(),lr=0.001)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer,patience=5,factor=0.1,mode='max')
lossf=nn.CrossEntropyLoss()
scoref=MulticlassF1Score(num_classes=len(classes))

In [13]:
print(model)

CNNmodel(
  (in_layer): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (h_layer): ModuleList(
    (0): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride

In [14]:
#summary(model,input_size=feature.permute(0,3,1,2).shape)

In [15]:
TRAIN=utils.Train_val(trainDL,testDL,model,optimizer,lossf,scoref)

In [16]:
EPOCH=20
history=TRAIN.train(EPOCH,scheduler,'1')

RuntimeError: Given groups=1, weight of size [16, 3, 3, 3], expected input[64, 32, 32, 3] to have 3 channels, but got 32 channels instead

[5/20]  
train loss 1.1080276055055327, train score 0.5532612294359079   
test loss 1.1347213983535767, test score 0.5854166746139526   
scheduler.num_bad_epochs 0/5       
틀린걸 고쳤으나......................................................ㅜ

In [ ]:
pp=utils.Plot_History(history)
pp.draw()